# Gradio UI for LLM 

This project will build a User Interface for a llm project

In [1]:
import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display 
from openai import OpenAI

In [4]:
# importing gradio 
import gradio as gr

/opt/anaconda3/envs/llms/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
#initializing constants
ollama_url = 'http://localhost:11434/v1'
llama_api_url = "http://localhost:11434/api/chat"
Headers = {"Content-Type": "application/json"}
Model = "llama3.1"

In [8]:
# A generic system message
system_message = "You are a helpful assistant"

In [53]:
# initialize the ollama local model

def message_llama(prompt):
    ollama = OpenAI(base_url=ollama_url, api_key="ollama")

    message = [
        {"role":"system", "content":system_message},
        {"role":"user", "content": prompt}
    ]
    completion = ollama.chat.completions.create(
        model=Model,
        messages = message
    )

    return completion.choices[0].message.content


In [54]:
message_llama("What is today's date?")

"# Today's Date: \nUnfortunately, I'm a large language model, I don't have real-time access to the current date. However, you can share your location or timezone with me, and I'll do my best to provide you with the current date.\n\nAlternatively, if you need help with a specific task or calculation that involves dates, feel free to share more information, and I'll be happy to assist you!"

# Create a UI using Gradio

In [13]:
#create a test function to shout (uppercase)
def shout(text):
    return text.upper()

In [18]:
# fn=function, input, output
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", allow_flagging="never").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [22]:
# A tailored system message that returns in Markdown
system_message = "You are a helpful assistant that responds in markdown"

# creating a Gradio UI for llama llm
gr.Interface(fn=message_llama,
            inputs=[gr.Textbox(label="Your Message:", lines=6)],
            outputs=[gr.Textbox(label="Chat Output:", lines=10)],
            flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [55]:
# Create a stream function for llm response 
# Yeild -> Generators

def stream_llama(prompt):
    ollama = OpenAI(base_url=ollama_url, api_key="ollama")

    message = [
        {"role":"system", "content":system_message},
        {"role":"user", "content": prompt}
    ]
    stream = ollama.chat.completions.create(
        model=Model,
        messages = message,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
    yield result


## Generators (the "yield" keyword) 
Why Do We Need Generators?
- Memory Efficient : Handle large or infinite data without loading everything into memory.
- No List Overhead : Yield items one by one, avoiding full list creation.
- Lazy Evaluation : Compute values only when needed, improving performance.
- Support Infinite Sequences : Ideal for generating unbounded data like Fibonacci series.
- Pipeline Processing : Chain generators to process data in stages efficiently.


In [29]:
# A tailored system message that returns in Markdown
system_message = "You are a helpful assistant that responds in markdown format"

# creating a Gradio UI for llama llm
gr.Interface(fn=stream_llama,
            inputs=[gr.Textbox(label="Your Message:", lines=6)],
            outputs=[gr.Markdown(label="Chat Output:")],
            flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


# Adding Deepseek-r1 8b 

Notes:
- Ollama runs its API server on http://localhost:11434/v1 by default.
- You don’t need to change that whether you’re calling deepseek-r1:8b, llama3.1, or any other model you’ve pulled.
-The only thing that changes is the model name you pass in your request.

In [66]:
# Create a stream function for DeepSeekllm response 

def stream_deepseek(prompt):
    ollama = OpenAI(base_url=ollama_url, api_key="ollama")

    message = [
        {"role":"system", "content":system_message},
        {"role":"user", "content": prompt}
    ]
    stream = ollama.chat.completions.create(
        model="deepseek-r1:8b",
        messages = message,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
    yield result


In [68]:
# initialize the ollama local model

def message_deepseek(prompt):
    ollama = OpenAI(base_url=ollama_url, api_key="ollama")

    message = [
        {"role":"system", "content":system_message},
        {"role":"user", "content": prompt}
    ]
    completion = ollama.chat.completions.create(
        model="deepseek-r1:8b",
        messages = message
    )

    return completion.choices[0].message.content


In [63]:
# create model functions for Gradio
def stream_model(prompt, model):
    if model=="Llama":
        result = stream_llama(prompt)
    elif model=="Deepseek":
        result = stream_deepseek(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [64]:
gr.Interface(
    fn=stream_model, 
    inputs=[gr.Textbox(label="Your Message:"), gr.Dropdown(["Llama","Deepseek"], label="Select Model", value="Llama")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode='never').launch()

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.
